In [2]:
import re
import pandas as pd 
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Functions for detecting intraocular lens model and power from free text operative notes

In [11]:
def iolpowerfeature(text): 
    try: 
        iolpower=re.findall('(?i)([1-9][0-9]?\.[0|5])0?[\s\-]*d[iopter]?', text)[0]
    except: 
        try: iolpower=re.findall('(?i)[\s\+\-]([1-9][0-9]?)[\s\-]*diopter', text)[0]+".0"
        except: 
            try: iolpower=re.findall('(?i)[\s\+\-]([1-9][0-9]?)[\s\-]*D\s', text)[0]+".0"
            except: 
                try: iolpower=re.findall('[\s\+\-]([1-9][0-9]?\.[0|5])', text)[0]
                except: iolpower=np.nan
    return float(iolpower)

def iolmodelfeatures(text): 
    alconiol=re.findall('(?i)([SMLC][ANVZXTC][1-7DA][5A10C34][TDWABMPU][0-9FSTCAMDO])', text)
    amoiol=re.findall('(?i)(Z[CX]T[1-6][0257][05]|Z[CKLMX][BAR][0O]{2}|Z9002|ZA9003|AR40[ME]|PCB[0O]{2}|NXG1|Tecnis CL)', text)
    bnliol=re.findall('(?i)(AO[12]UV|Crystalens|MX60|enVista|AT50T|BL1AT|Trulign|M160L|AO60|Akreos MICS|Akreos AO|Akreos|LI61A0|LI61SE|SofPort|iSert 2[35][01])', text)

    if len(alconiol)>0: 
        iolpower=iolpowerfeature(text) 
        return ("Alcon", alconiol[0].upper(), iolpower)
    elif len(amoiol)>0: 
        iolpower=iolpowerfeature(text) 
        if amoiol[0].upper()=="ZCBOO": 
            amoiol[0]="ZCB00" #fix the common zcboo typo 
        return ("AMO", amoiol[0].upper(), iolpower)
    elif len(bnliol)>0: 
        iolpower=iolpowerfeature(text)
        return ("Bausch and Lomb", bnliol[0].upper(), iolpower)
    else: 
        return (None, None, iolpowerfeature(text))

IOL dioptric power is extracted from notes using regular expressions capturing numeric expressions in the form of plausible IOL powers-  e.g., 21.5 or 16.0 etc. potentially followed by “diopter” or an indicator that the preceding numeric expression refers to dioptric power, e.g. “21.5 D” or “16.0 diopter”, but not “7.0mm”. IOL model codes from the major manufacturers of Alcon, AMO, and Bausch and Lomb are extracted similarly from regular expressions capturing plausible IOL model codes, e.g. “SN60WF”, “SA60AT”, or “ZCB00”, “ZXR00”, etc.

List of IOL's recognized: 

Alcon		
	ReSTOR	SV25T0
	ReSTOR	SN6AD1
	ReSTOR	SN6AD3
	ReSTOR	MN6AD1
	ReSTOR	SV25T2
	ReSTOR	SV25T3
	ReSTOR	SV25T4
	ReSTOR	SV25T5
	ReSTOR	SV25T6
	ReSTOR	SND1T2
	ReSTOR	SND1T3
	ReSTOR	SND1T4
	ReSTOR	SND1T5
	ReSTOR	SND1T6
	AcrySof IQ	SN6AT2
	AcrySof IQ	SN6AT3
	AcrySof IQ	SN6AT4
	AcrySof IQ	SN6AT5
	AcrySof IQ	SN6AT6
	AcrySof IQ	SN6AT7
	AcrySof IQ	SN6AT8
	AcrySof IQ	SN6AT9
	AcrySof IQ	SN60WF
	AcrySof IQ	SN6CWS
	AcrySof Natural	SN60AT
	AcrySof Natural	SA60AT
	AcrySof Multipiece	MN60AC
	AcrySof Multipiece	MA30AC
	AcrySof Multipiece	MA60AC
	AcrySof Multipiece	MA60BM
	AcrySof Multipiece	MN60MA
	AcrySof Multipiece	MA60MA
		MZ40BD
		LX10BD
		MZ30BD
		MZ60BD
		MC50BD
		CZ70BD
		MZ60MD
		MZ60PD
		MTA3UO
		MTA4UO
		MTA5UO
        
AMO/J&J Vision 	
	Tecnis	ZCT150
	Tecnis	ZCT225
	Tecnis	ZCT300
	Tecnis	ZCT400
	Tecnis	ZCT450
	Tecnis	ZCT525
	Tecnis	ZCT600
	Tecnis	ZCB00
	Tecnis	ZKB00
	Tecnis	ZLB00
	Tecnis	ZMB00
	Tecnis	ZMA00
	Symfony	ZXR00
	Symfony	ZXT150
	Symfony	ZXT225
	Symfony	ZXT300
	Symfony	ZXT375
	Tecnis iTec	PCB00
	Tecnis	ZA9003
	Tecnis CL	Z9002
	Sensar	AR40M
	Sensar	AR40E
	Sensar	AR40e
	ReZoom	NXG1
    
Bausch and Lomb 	
	Crystalens AO	AO1UV
	Crystalens AO 	AO2UV
	enVista	MX60
	Trulign	BL1UT
	Trulign	BL1UT
	Trulign	BL1UT
	Trulign	AT50T
	Trulign	AT50T
	Trulign	AT50T
	Trulign	BL1AT
	Trulign	BL1AT
	Trulign	BL1AT
	iSert	230
	iSert	231
	iSert	250
	iSert	251
	Akreos MICS	M160L
	Akreos AO	AO60
	SofPort	LI61A0
	SofPort	LI61SE


# Examples

## Example simple code

In [8]:
#example simple code for how this works 
iolpowerfeature("1. Descemet's membrane, 2.5 mL injected rotated 90 degrees used a 7.0 malyugin ring model 123345.50 model Tecnis Z9002 +23 diopters ")
iolmodelfeatures("SN60WF 23.0D")
iolmodelfeatures("ZCB00 19.0D")
iolmodelfeatures("MTA4UO 14.0D")

23.0

('Alcon', 'SN60WF', 23.0)

('AMO', 'ZCB00', 19.0)

('Alcon', 'MTA4UO', 14.0)

## Example code for how this might work on a pandas dataframe containing a column of operative reports

In [9]:
#example code for how this might work on a pandas dataframe containing a column of operative reports 
df=pd.DataFrame(["SN60WF 23.0", 
 "ZCB00 19.0D",
"MTA4UO 14.0 diopter"], columns=["note"])
df

,note
0,SN60WF 23.0
1,ZCB00 19.0D
2,MTA4UO 14.0 diopter


In [10]:
df["iolbrand"], df["iolmodel"], df["iolpower"] = zip(*df["note"].apply(iolmodelfeatures))
df

,note,iolbrand,iolmodel,iolpower
0,SN60WF 23.0,Alcon,SN60WF,23.0
1,ZCB00 19.0D,AMO,ZCB00,19.0
2,MTA4UO 14.0 diopter,Alcon,MTA4UO,14.0
